<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Machine Learning
### Sommersemester 2023
Prof. Dr. Heiner Giefers

## Entscheidungsgrenzen

In dieser Aufgabe sollen die Entscheidungsgrenzen eines logistischen Regressionsmodells für einen synthetisch generierten Datensatz gefunden werden.

Führen Sie dazu die Aufgaben **(a)** bis **(c)** durch.

In [ ]:
from sklearn.datasets import make_blobs
import numpy as np
import matplotlib.pyplot as plt


# generating two-class dataset
X, y = make_blobs(n_samples=100, centers=2, n_features=2, center_box = (-5, 5))


plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

**(a)** Teilen Sie den Datensatz auf (70% Training, 30% Test):
- `X_train`: training dataset
- `X_test`: test dataset
- `y_train`: training labels
- `y_test`: test labels

In [ ]:
X_train, X_test, y_train, y_test = [None]*4
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test Cell
#----------

assert np.vstack((X_train, X_test)) in X and np.hstack((y_train, y_test)) in y
assert y_train.size/y.size == 0.7

**(b)** Verwenden Sie die *sklearn*-Klasse `LogisticRegression` um ein Modell für den Datensatz zu bilden. Trainieren Sie das Modell mit den oben festgelegten Trainingsdaten.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Test Cell
#----------

assert type(logreg) == LogisticRegression
assert logreg.intercept_, 'Trainieren Sie das Modell mit den Daten!'

Wir visualisieren nun den Datensatz um darzustellen, wie gut unser Modell klassifiziert:

In [ ]:
# Plotting decision regions
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),np.arange(y_min, y_max, 0.01))

Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.Spectral)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)
plt.xlabel(r"x", fontsize=14)
plt.ylabel(r"y", fontsize=14)
plt.show()

**(c)** Testen Sie die Genauigkeit (*accuracy*) des Modells mit den Testdaten:

In [ ]:
y_pred = None
acc_test = None

# YOUR CODE HERE
raise NotImplementedError()

print("accuracy: %.2f%%" % acc_test)

In [ ]:
# Test Cell
#----------

assert y_pred.shape == y_test.shape
### BEGIN HIDDEN TEST
assert acc_test == 100-np.sum(np.abs(y_pred-y_test))*100/len(y_pred)
### END HIDDEN TEST

## Multiklassen-Klassifikation 

Die logistische Regression liefert uns Ergebnisse für binäre Zielvariablen.
Oftmals wollen wir aber mehr als 2 Klassen unterscheiden.

Eine Möglichkeit, um Multiklassen-Klassifikation mit logistischer Regression umzusetzen, ist die sogenannte *One-vs-all Klassifikation*.
Dabei werden für `n` Klassen `n` separate, binäre Klassifikationsprobleme definiert, bei denen jeweils nur die betrachtete Klasse den Zielwert `1` zugeteilt bekommt, und für alle anderen Klassen der Zielwert `0` angenommen wird.

Scikit-Learn unterstüzt Multiklassen-Klassifikation in der Klasse `LogisticRegression` über den Parameter `multi_class`.
Setzt man `multi_class="ovr"` benutzt die führt die Funktion `fit` je eine logistische Regression für jedes Label nach dem *one-vs-all* (oder auch *one-vs-rest*, ovr) Prinzip aus.

In der folgenden Code-Zelle erzeugen wir 3 Punktwolken.
Alle Punkte einer "Wolke" sollen zu einer bestimmten Klasse gehören.

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from pandas import DataFrame

X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=10)
df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))

colors = {0:'red', 1:'blue', 2:'green'}
markers = {0:'o', 1:'x', 2:'^'}
fig, ax = plt.subplots()
grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, marker=markers[key], color=colors[key])
    
plt.legend(loc='upper right', prop={'size': 12})
plt.savefig("LogistischeRegression20.png",transparent=True, dpi=300)
plt.show()

Nun wenden wir ein logistisches Regressionsmodell auf die Datenbasis an.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

model = LogisticRegression(solver='lbfgs',multi_class="ovr")

model.fit(X_train,y_train)

model.intercept_ , model.coef_


Der folgende Graph zeigt die Entscheidungsgrenzen für das Klassifikationsmodell.
Alle Punkte innerhalb eines Bereiches werden der jeweiligen Klasse zugeordnet.

In [ ]:
# Plotting decision regions
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),np.arange(y_min, y_max, 0.01))

Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

fig, ax = plt.subplots()

colors = {0:'red', 1:'blue', 2:'green'}
markers = {0:'o', 1:'x', 2:'^'}

ax.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.Spectral)

grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, marker=markers[key], color=colors[key])

plt.xlabel(r"x", fontsize=14)
plt.ylabel(r"y", fontsize=14)
plt.show()